You can follow along and play with this notebook by clicking the badge below

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jasongfleischer/UCSD_COGS118A/blob/main/Notebooks/Lecture_08_metrics.ipynb)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['font.size'] = '16'

import seaborn as sns
sns.set_style('white')

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

from sklearn.metrics import zero_one_loss, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve
from sklearn.metrics import confusion_matrix, plot_confusion_matrix


The UCI ML Breast Cancer Wisconsin (Diagnostic) dataset is a binary classificiation task generated from microscopic images of 569 biopsy samples.  The class of each sample is either malignant or benign cancer. The variables are statistical summaries of attributesmeasured from the cell nuclei of each image. For instance, attributes include the area of a cell nucleus, shape measurements (like concavity of nucleus), grayscale texture, and many others. Each such attribute is measured for each cell nucleus in the image. For each attribute three statistical summaries are calculated to create the features: the mean, standard error, and worst (largest) measurement made across all the cell nuclei in the image. 

For more details see https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset

In [ ]:
data  = load_breast_cancer()
df = pd.DataFrame( data['data'], columns=data['feature_names'] )

fig = plt.figure(figsize = (16,12))
ax = fig.gca()
df.hist(ax = ax);
plt.tight_layout();

In [ ]:
lnames = data['target_names']
labels = [ lnames[1] if el else lnames[0] for el in data['target'] ]

sns.countplot(labels);

# Data summary
You can see the 30 data features (variables) above. Most of the histograms show probably skew-normal distributions; a couple of them are exponential looking, one is clearly bimodal. The measurements span 5 orders of magnitude across the features.

# Our default classifier
Let's use a decision stump.  Why?  

Well it's about as dumb a classifier as we can use, and this is supposed to be quite an easy classification problem so let's test that. 

Oh yeah, and it's also the only classifier we've shown you so far that actually is implementable in sklearn 

In [ ]:
# to get a stump, just create a Decision Tree of depth 1 
stump = DecisionTreeClassifier(max_depth=1)
stump

There is one difference in sklearn's DTstump compared to what we've talked about so far. This tree algorithm does NOT minimize misclassification error as we've talked about so far. Instead if minimizes something called the Gini impurity.  Don't worry that we haven't covered this yet... we will by the end of the quarter. 

The basic view of what's going on is this: The stump tries to reduce the "impurity" of the leaves of the tree by making them mostly-just-one-class-when-sorting-training-data. In this way minimizing Gini impurity is also minimizing misclassification error, but using Gini is going to force the stump to break ties between certain choices that have identical misclassification error in a systematic way. We will cover this in a later class, but if you'd like a preview of this take a look at: https://sebastianraschka.com/faq/docs/decision-tree-binary.html

## Question:
Why do we NOT need to use a `StandardScaler()` on this data if we are using a decision stump (or a more full decision tree)? 

# Training our stump

Let's take a random 1/3 of our data as test set and fit the stump on the rest.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( data['data'], data['target'], 
                                                    test_size=0.33, random_state=42)

In [ ]:
stump.fit(X_train, y_train)

y_pred_train = stump.predict(X_train)
y_pred_test = stump.predict(X_test)

OK, how did our tree choose to split malignant vs benign cancers? It chose to split on the mean number of concave points per cell nucleus at a threshold of 0.051. I think that's the mean number of dents in a nucleus, but TBH I'm not sure cause I didn't read the original paper :)

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.gca()
out = plot_tree(stump, fontsize=12, ax=ax);
# this stuff below is only necessary if you have an sklearn version < 0.24 
# theres a bad interaction between the graphic style of seaborn and plot_tree
# that erases the arrows; this is just to force a redraw
for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor('red')
        arrow.set_linewidth(3)

Looking at the stumpy tree above, you can see that `value` shows how many of each class are in each node of the tree.  The root (ie, the whole training set) has 145 negative class and 236 positive class samples. After splitting on the threshold we get the leftmost leaf having 12 negative class and 221 positive class samples; the rightmost leaf has 133 neg vs 15 pos. So the leaves are pretty pure.

Also this kinda diagram shows you the confusion matrix.  Other kinds of classifiers don't have this built-in like the decision tree viz above. In those cases you might want to calculate it using one of the two functions below.  

Note the for binary classification the confusion matrix shows you 

|                 |                |
|-----------------|----------------|
| True Negatives | False Positives |
| False Negatives | True Positives |

For multiclass (>2) situations that isn't true anymore, but in multiclass situations the confusion matrix is super useful for understanding the systematic errors the algo is making.


In [ ]:
# Here are two equivalent ways to get a confusion matrix

# the plot version takes the classifier and inputs+labels (it generates the predictions itself)
plot_confusion_matrix(stump, X_train, y_train );

# the text version takes the true labels and the predictions AND THE ORDER OF ARGUMENTS MATTERS!! don't mess it up
confusion_matrix(y_train, y_pred_train)

# better yet, you could use named arguments like below to avoid the order problem ;)
# confusion_matrix(y_true=y_train, y_pred=y_pred_train)

# Metrics

Here's some metrics we talked about during lecture that we are going to implement below to find out about the generalization performance of our decision stump.

1. Zero-one loss; this is just another name for misclassification error 
1. Accuracy score; 1 - misclassification error
1. Precision score; How precisely does the classifier identify positive cases? True positives / all predicted positives
1. Recall score; How many of the actual positive cases are recalled by the classifier? True positives / all actually positive cases
1. ROC AUC; the area under the receiver operating characteristics curve. ROC AUC close to 1 is a great performing algorithm, close to 0.5 is chance performance level for binary classification with balanced class sample sizes.  A further discussion of what the heck this curve actually is will be taken up below ;)


In [ ]:
metrics_to_try = [ zero_one_loss, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score ]
for m in metrics_to_try:
    print('Metric {}: training={}, test={}\n'.format( m.__name__,
                                                    m(y_train, y_pred_train),
                                                    m(y_test, y_pred_test) ))


As you can see even the relatively poor stump algorithm does quite well on this task.  Only 11% of test set items are wrong. 

Stop here and ask yourself some questions: 
- what pattern do we see between training set and test set performance? Why?
- Is this algorithm likely underfit or overfit? Why?
- What does this algorithm do better in, precision or recall?  What does that imply?

Next I'd like to take a look at some cool plots. But the decision stump as implemented by sklearn is useless for showing you ROC & Precision-recall curves. If you'd like to see how lame they are go ahead and try to implement it yourself using what you learn below.

So instead we are going to bring out a classifier you haven't seen yet (but will soon!): Logistic Regression.
I know you have no idea what the loss function is, or what this thing does yet.  Let's ignore that for now, just think of it as a magic ML box that draws a linear decision boundary to perform binary classification. 

Unlike with decision stumps, this time those funky features, where some are 5 orders of magnitude larger than others, need to be standardized or this algorithm won't be able to deal well with it.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

logreg = LogisticRegression() 

# we fit the scaler only on the training set, just like with the algorithm
# if we fit the scaler on ALL data test set... that's leaking information about the test set into the training set
# if we refit the scaler on the test data before applying it there... well that's just making it harder to generalize well :)

scaler = StandardScaler()
scaler.fit(X_train) # calculate the mean of each feature

X_train_s = scaler.transform(X_train) # turn each sample's measure into how many stddevs away from mean it is
X_test_s = scaler.transform(X_test)

logreg

In [ ]:
logreg.fit(X_train_s, y_train)

y_pred_train = logreg.predict(X_train_s) #obvs we will use the scaled version of the data!
y_pred_test = logreg.predict(X_test_s)

In [ ]:
metrics_to_try = [ zero_one_loss, accuracy_score, precision_score, recall_score, roc_auc_score,  f1_score]
for m in metrics_to_try:
    print('Metric {}: training={}, test={}\n'.format( m.__name__,
                                                    m(y_train, y_pred_train),
                                                    m(y_test, y_pred_test) ))


Stop here and ask yourself some questions: 
- Do we see the same pattern of difference between training and test set performance as we did for the decision stump? 
- Which classifier has better test set performance, logistic regression or the stump?
- Which classifier has better (less) generalization error? 
- What else do you notice about these results?

OK let's talk about Precision - Recall and Receiver Operating Characteristic curves.  These curves are useful for understanding many different aspects of classifier performance. At their heart, these methods rely on your classifier being able to be adjusted by moving the decision boundary back and forth, such that we can tradeoff how often we make one kind of mistake (false positives) vs another (false negatives).

But rather than relying on my poor words I want you to RIGHT CLICK + OPEN LINK IN NEW WINDOW on the animation below RIGHT NOW!  This link will take you to a wonderful animation-based tutorial by Dariya Sydykova that will help you understand the important concepts of ROC and PR curves. 

[![Link to ROC animation tutorial](https://raw.githubusercontent.com/dariyasydykova/open_projects/master/ROC_animation/animations/cutoff.gif)](https://github.com/dariyasydykova/open_projects/tree/master/ROC_animation)


In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.gca()
plot_precision_recall_curve(logreg,X_test_s,y_test, ax=ax);


So here we can see that LogReg on this task is super good. Even when we get to perfect recall (classifier correctly calls all malignant cancers as malignant) we are not making very many false positives (we don't label very many benign cancers as malignant; ie, the precision is still very high).  

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.gca()
plot_roc_curve(logreg,X_test_s,y_test, ax=ax);
#plt.ylim(bottom=0.0);

The ROC curve backs this up... it very quickly goes up to perfect true positive rate (y-axis) with very few false positives (x-axis). As mentioned above the performance on this task is excellent, and thus the curve is close to the top left of the graph, and the AUC is very close to 1.0